In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import Image

import os, sys, re, datetime, time
from pathlib import Path

pj_dir = Path(os.getcwd()).parents[1]
data_dir = pj_dir/'data'
img_dir = pj_dir/'images'
src_dir = pj_dir/'src'
sys.path.append(str(src_dir))

from matplotlib import pyplot as plt
import japanize_matplotlib
import seaborn as sns
plt.style.use("bmh")
import numpy as np
import pandas as pd
import dask.dataframe as dd

from tqdm import tqdm_notebook
from dotenv import load_dotenv

In [2]:
import matplotlib
matplotlib.rcParams["figure.figsize"] = (16, 4)
plt.rcParams["font.family"] = "IPAexGothic"
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
import copy

In [4]:
import jpholiday

In [5]:
import requests

In [6]:
pd.set_option("display.max_rows", 100)

# ロード

In [8]:
dfall = pd.read_pickle(data_dir/'kaizen_slack/all_messages.pickle')
df_channel = pd.read_pickle(data_dir/'kaizen_slack/channels.pickle')
df_member = pd.read_pickle(data_dir/'kaizen_slack/members.pickle')
df = pd.read_pickle(data_dir/'kaizen_slack/messages.pickle')

In [10]:
not_message_types = ['channel_join', 'channel_leave', 'channel_topic', 'channel_archive', 'channel_purpose', 'sh_room_created', 'channel_name', 'pinned_item', 'reminder_add', 'app_conversation_join']
df = df[~df['subtype'].isin(not_message_types)]

In [11]:
df['text_trimed'] = df['text'].str.replace(r'<\S+>', '').str.replace(r':\S+:', '').str.replace('\n', '')

In [12]:
import MeCab

In [13]:
from sklearn.manifold import TSNE

# Doc2Vec

In [16]:
from gensim import models
from gensim.models.doc2vec import TaggedDocument

INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


In [15]:
from gensim.models import word2vec

INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


In [16]:
model = word2vec.Word2Vec(window=1, min_count=1)

In [17]:
df.shape

(376096, 53)

In [18]:
doc = ' '.join(df['text_trimed'].values.tolist())

In [26]:
def split_df(df, n):
    for idx in range(0, df.shape[0], n):
        yield df.iloc[idx:idx + n, :]

In [53]:
def split_into_words(doc):
    mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
    lines = mecab.parse(doc).splitlines()
    words = []
    for line in lines:
        chunks = line.split('\t')
        if len(chunks) > 3 and (chunks[3].startswith('動詞') or chunks[3].startswith('形容詞') or (chunks[3].startswith('名詞') and not chunks[3].startswith('名詞-数'))):
            words.append(chunks[0])
    return words

In [54]:
words = []

for i, row in df.iterrows():
    tmp_words = split_into_words(row['text_trimed'])
    words.append(tmp_words)

In [59]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

In [60]:
model = Word2Vec(words, size=100, window=5, min_count=1, workers=4)

INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 105685 words, keeping 10547 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 187020 words, keeping 14261 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #30000, processed 291905 words, keeping 18029 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #40000, processed 399301 words, keeping 20699 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #50000, processed 526440 words, keeping 22339 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #60000, processed 624872 words, keeping 24889 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #70000, processed 761931 words, keeping 28136 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #80000, processed 889971 words, keeping 3

INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 4 : training on 4700548 raw words (4257950 effective words) took 3.5s, 1204481 effective words/s
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 31.97% examples, 1260523 words/s, in_qsize 6, out_qsize 1
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 59.65% examples, 1299866 words/s, in_qsize 8, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 86.85% examples, 1261674 words/s, in_qsize 7, out_qsize 0
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 3 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 5 : training on 4700548 raw words 

In [62]:
model.save(str(data_dir/'kaizen_slack/w2v.model'))

INFO:gensim.utils:saving Word2Vec object under /Users/yutaro_ikeda/.ghq/github.com/ikedaosushi/python-sandbox/data/kaizen_slack/w2v.model, separately None
INFO:gensim.utils:not storing attribute vectors_norm
INFO:gensim.utils:not storing attribute cum_table
INFO:gensim.utils:saved /Users/yutaro_ikeda/.ghq/github.com/ikedaosushi/python-sandbox/data/kaizen_slack/w2v.model


In [63]:
model.wv.most_similar('keiji')

INFO:gensim.models.keyedvectors:precomputing L2-norms of word weight vectors
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('yasu', 0.9081540107727051),
 ('jimbo', 0.8986175656318665),
 ('kogure', 0.8938308954238892),
 ('andy', 0.8859750032424927),
 ('sakai', 0.8749969005584717),
 ('masao', 0.8628397583961487),
 ('banin', 0.8570758104324341),
 ('nishiya', 0.855423092842102),
 ('shino', 0.8463077545166016),
 ('榊原', 0.8343085050582886)]

In [68]:
model.wv.most_similar('kawabe')

/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('takus', 0.8530447483062744),
 ('watabe', 0.8480912446975708),
 ('inu', 0.8280203938484192),
 ('sue', 0.8273420333862305),
 ('keiji', 0.8154975175857544),
 ('もひ', 0.8071255683898926),
 ('andy', 0.8007967472076416),
 ('yu-ki', 0.8006150722503662),
 ('peco', 0.7980176210403442),
 ('ikeda', 0.7975234985351562)]

In [110]:
model.wv.most_similar(positive=["グーグル"], negative=[])

/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('HIS', 0.8339268565177917),
 ('鈴森', 0.8333430290222168),
 ('安江', 0.831693172454834),
 ('ゼクシィ', 0.8308569192886353),
 ('Rakuma', 0.8258123397827148),
 ('渡辺由美子', 0.8252314329147339),
 ('関口', 0.8242795467376709),
 ('大和田', 0.8239831328392029),
 ('ニューバランス', 0.8221687078475952),
 ('GMOインサイト株式会社', 0.8211828470230103)]